In [1]:
###日志 logger##20220825A######
import logging
import sys
import time

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s')

def make_logger(name, log_level=logging.INFO, log_file="log.txt", file_mode="a"): #w写 a追加
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(level=log_level)
        handler = logging.FileHandler(log_file, mode=file_mode)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        # logger.addHandler(handler)
    return logger

name = 'wsAcc'
curDate = time.strftime("%Y%m%d")  # debug  error info warning
logger = make_logger(name, log_level=logging.DEBUG, log_file= "../logs/%s_%s.log" % (name, curDate) )

version = "1.0.1"
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} ********** ')

Now: 1.0.1 : 2022-11-12 01:05:19 ********** 


In [2]:
###账户订阅 ws##20220825A######
import websocket
import time
import threading
import requests
import json

curTime = int(time.time())
listenKeyTime = curTime  #<60
pongTime = curTime  #<10

curDateTime = time.strftime("%Y-%m-%d %H:%M:%S")
version = '1.0.0'
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")}')

logger.info(f'--websocket.WebSocketApp--{version}---') # debug  error info warning

def on_open(wsapp):
    logger.info(f'--Starting--')

    def send_message():

        global curTime, listenKeyTime, pongTime 
        for i in range(5):
            # wsapp.send(f"Hello {i}")
            curTime = int(time.time())  
            if curTime - pongTime > 600:
                pongTime = curTime

            if curTime - listenKeyTime > 3000:
                listenKeyTime = curTime
                r = requests.put('http://http://testnet.binancefuture.com/fapi/v1/listenKey')  
            time.sleep(1) 
    threading.Thread(target=send_message).start()

def on_data(wsapp, frame_data, frame_opcode, frame_fin):
    logger.info(f'{frame_data = } \n{frame_opcode = } \n{frame_fin = }')

def on_message(wsapp, data):
    logger.info(f'{data = }')

def on_cont_message(wsapp, frame_data, frame_fin):
    logger.info(f'{frame_data = } \n{frame_fin = }')
    
def on_ping(wsapp, frame_data):
    logger.info(f'{frame_data = }') 
    wsapp.send("", websocket.ABNF.OPCODE_PONG) # 接收到 PING 数据帧后, 需要立即给服务端回复 PONG 数据帧

def on_pong(wsapp, frame_data):
    logger.warning(f'{frame_data = }') 
    pass
    
def on_error(wsapp, err):
    logger.error(f'{err = }') 

def on_close(wsapp, close_status_code, close_reason):
    logger.info(f'{close_status_code = } \n{close_reason = }')

def getListenKey():
    listenKey_url = "http://testnet.binancefuture.com/fapi/v1/listenKey"
    
    #  #ation126@hotmail.com
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"    # 密钥 API 密钥
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876" # 私钥 API 私钥

    # testnet@snapmail.cc  Test2022 15,000.00000000
    # api_key    = "40642311a14066133c8076db5472bfff3683d3d1c73e48ae4c6bfc0b405b1957"  # 密钥 API 密钥
    # secret_key = "dcff08b6bdbf37caa606e3c6eaa11c2d59d5dcb5f7b1cf7c959aa123e463a43b"  # 私钥 API 私钥

    headers = {"x-mbx-apikey":api_key}
    res = requests.post(listenKey_url,  headers=headers) #, verify=False, proxies=proxy data={},
    rt = json.loads(res.text)
    # print(f'{rt["listenKey"]}') 
    return rt["listenKey"]

ws_url = f'wss://stream.binancefuture.com/ws/{getListenKey()}'

def wssApp():
    global wsapp
    wsapp = websocket.WebSocketApp(ws_url, on_open=on_open, on_data=on_data, on_message=on_message, on_cont_message=on_cont_message, on_ping=on_ping, on_pong=on_pong, on_error=on_error, on_close=on_close )

    wsapp.run_forever(ping_interval=30, ping_timeout=10)  # wsapp.close()

# t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
# # t1.setDaemon(True)  #False 同 t1.join()
# t1. start()



Now: 1.0.0 : 2022-11-12 01:05:21


In [2]:
###行情订阅 ws##20220827A######
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np

worthDir = '/home/at/py/ipynb/'
dbfile = '%s/%s' % ('/mnt/e/milu/py', 'binance.db')

curTime = int(time.time())
listenKeyTime = curTime  #<60
pongTime = curTime  #<10

curDateTime = time.strftime("%Y-%m-%d %H:%M:%S")
version = '1.0.0'

symbol = pair = 'BTCUSDT'
contractType = 'PERPETUAL'
interval = '1m'

df_btcusdt_1m = pd.DataFrame()

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")}')

logger.info(f'--websocket.WebSocketApp--{version}---') #debug  error info warning

def on_open(wsapp):
    logger.info(f'--Starting--')

def on_data(wsapp, frame_data, frame_opcode, frame_fin):
    # logger.info(f'{frame_data = } \n{frame_opcode = } \n{frame_fin = }')
    pass

def on_message(wsapp, data):

    global df_btcusdt_1m

    logger.info(f'{data = }')
    dk = json.loads(data)
    if dk['ps'] == symbol and dk['ct'] == contractType and dk['k']['i'] == interval and dk['k']['x'] == True:
        dk_df = pd.DataFrame(dk['k'], index=[0])

        dk_df.drop(columns = ['i', 'f', 'L', 'x', 'B', ], inplace=True)
        dk_df.columns = ['openTime','closeTime','open','close','high','low','vol','numTrade','Amt','bidVol','bidAmt'] #,'ignore'

        pd.to_numeric([dk_df['open'], dk_df['high'], dk_df['low'], dk_df['close'], dk_df['vol'], dk_df['Amt'], ], errors='coerce', downcast='float')
        pd.to_numeric([dk_df['openTime'], dk_df['closeTime'], dk_df['numTrade'] ], errors='coerce', downcast='integer')

        dk_df['openDateTime'] = pd.to_datetime(dk_df['openTime'], unit="ms", origin="1970-01-01 08:00:00") 
        dk_df['closeDateTime'] = pd.to_datetime(dk_df['closeTime'], unit="ms", origin="1970-01-01 08:00:00") 
        # dk_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

        dk_df.sort_index(axis=1, ascending=True, inplace=True)

        print(f'{dk_df = }  \n')

        if  not df_btcusdt_1m.empty and dk['k']['t'] in df_btcusdt_1m['openTime'].values and dk['k']['T'] in df_btcusdt_1m['closeTime'].values:
            df_btcusdt_1m.sort_index(axis=1, ascending=True, inplace=True)
            df_btcusdt_1m.loc[ df_btcusdt_1m['closeTime'] == dk['k']['T'] ] = dk_df.to_numpy()
        else:
            df_btcusdt_1m = pd.concat([df_btcusdt_1m, dk_df ], ignore_index=True)

        df_btcusdt_1m.to_pickle('%s/%s.pkl' % (worthDir, "btcusdt_1m" ) )
        # df_btc = pd.read_pickle('%s/%s.pkl' % (worthDir, "btcusdt_1m" ) )

def on_cont_message(wsapp, frame_data, frame_fin):
    # logger.info(f'{frame_data = } \n{frame_fin = }')
    pass
    
def on_ping(wsapp, frame_data):
    logger.info(f'{frame_data = }') 
    wsapp.send("", websocket.ABNF.OPCODE_PONG) # 接收到 PING 数据帧后, 需要立即给服务端回复 PONG 数据帧

def on_pong(wsapp, frame_data):
    # logger.warning(f'{frame_data = }') 
    pass
    
def on_error(wsapp, err):
    logger.error(f'{err = }') 
    pass

def on_close(wsapp, close_status_code, close_reason):
    logger.info(f'{close_status_code = } , {close_reason = }')
    pass

def getListenKey():
    listenKey_url = "http://testnet.binancefuture.com/fapi/v1/listenKey"
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    headers = {"x-mbx-apikey":api_key}
    res = requests.post(listenKey_url,  headers=headers) #, verify=False, proxies=proxy data={},
    rt = json.loads(res.text)
    # print(f'{rt["listenKey"]}') 
    return rt["listenKey"]

# ws_url = f'wss://stream.binancefuture.com/ws/{getListenKey()}'
# ws_url = f'wss://fstream.binance.com/ws/btcusdt_perpetual@continuousKline_1m'  #ok
# ws_url = f'wss://fstream.binance.com/ws/!markPrice@arr' 
# ws_url = f'wss://fstream.binancefuture.com/ws/btcusdt_perpetual@continuousKline_1m' #ok
# ws_url = f'wss://fstream.binancefuture.com/ws/bnbusdt@aggTrade'

url = f'wss://fstream.binancefuture.com'
streamName = f'{symbol.lower()}_{contractType.lower()}@continuousKline_{interval}'
ws_url = f'{url}/ws/{streamName}'

def wssApp():
    global wsapp
    wsapp = websocket.WebSocketApp(ws_url, on_open=on_open, on_data=on_data, on_message=on_message, on_cont_message=on_cont_message, on_ping=on_ping, on_pong=on_pong, on_error=on_error, on_close=on_close )

    wsapp.run_forever(ping_interval=30, ping_timeout=10)  # wsapp.close()

# t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
# # t1.setDaemon(True)  #False 同 t1.join()
# t1. start()


Now: 1.0.0 : 2022-09-06 23:53:51


In [3]:
t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
t1. start()

In [ ]:
# sys.exit('退出')
t1.is_alive()
wsapp.close()
t1.is_alive() #False
print(f'{t1.is_alive() = } , {t1.ident = } , {t1.name = } , ')

In [ ]:
import pandas as pd
import numpy as np
import time

worthDir = '/home/at/py/ipynb/'

In [ ]:
###余额 balance###可用###分析###
df_bal = pd.read_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )
df_bal_diff = df_bal[['availableBalance','balance','maxWithdrawAmount','updateDate','updateTime']].diff(axis=0, periods=1)
df_bal_diff.iloc[0] = 0
df_bal_diff['queryTime'] = df_bal['queryTime'] 
display(df_bal)
print('')
display(df_bal_diff)

df = df_bal
# print(f'保证金占用 : { float(df["balance"]) - float(df["availableBalance"])  } ')  #

# "accountAlias": "SgsR",    // 账户唯一识别码
# "asset": "USDT",        // 资产
# "balance": "122607.35137903",   // 总余额
# "crossWalletBalance": "23.72469206", // 全仓余额
# "crossUnPnl": "0.00000000"  // 全仓持仓未实现盈亏
# "availableBalance": "23.72469206",       // 下单可用余额
# "maxWithdrawAmount": "23.72469206",     // 最大可转出余额
# "marginAvailable": true,    // 是否可用作联合保证金
# "updateTime": 1617939110373

print( f'{time.strftime("%Y-%m-%d %H:%M:%S")} *****余额 balance###可用###分析***** ') 


In [ ]:
###账户 account###可用###分析###
df_acc = pd.read_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )
df_acc_diff = df_acc[['availableBalance','crossUnPnl','crossWalletBalance','maintMargin','marginBalance','maxWithdrawAmount','openOrderInitialMargin','positionInitialMargin','unrealizedProfit','walletBalance','updateTime']].diff(axis=0, periods=1)
df_acc_diff.iloc[0] = 0
df_acc_diff['queryTime'] = df_acc['queryTime'] 
display(df_acc)
print('')
display(df_acc_diff)


df = df_acc.iloc[-1]

print()
print(f'钱包余额 walletBalance = 保证金余额 marginBalance - 未实现盈亏 unrealizedProfit : {df["walletBalance"]} =  { float(df["marginBalance"]) - float(df["unrealizedProfit"])  } ')  #ok

# print(f'? 可用余额 availableBalance = 保证金余额 marginBalance - 保证金占用 initialMargin : {df["availableBalance"]} =  { float(df["marginBalance"]) - float(df["availableBalance"])  } ')  #


print(f'保证金余额 marginBalance : { float(df["marginBalance"]) } , 可用余额 availableBalance  : {float(df["availableBalance"]) } , 保证金占用 initialMargin : {float(df["initialMargin"])  } ') 
print(f'保证金占用 差额 : { float(df["marginBalance"]) - float(df["availableBalance"]) - float(df["initialMargin"])  } ')  #
# print(f'保证金占用  = 保证金余额 marginBalance - 可用余额 availableBalance  : { float(df["marginBalance"]) - float(df["availableBalance"]) - float(df["initialMargin"])  } ')  #

print(f'当前所需起始保证金 initialMargin = 持仓所需起始保证金 positionInitialMargin + 当前挂单所需起始保证金 openOrderInitialMargin : {df["initialMargin"]} =  { float(df["positionInitialMargin"]) + float(df["openOrderInitialMargin"])  } ')  #ok

df = df_acc.iloc[-1]

print(f'? 可用余额 availableBalance = 保证金余额 marginBalance - 保证金占用 initialMargin : {df["availableBalance"]} =  { float(df["marginBalance"])  - float(df["positionInitialMargin"])   } ')  #err + float(df["maintMargin"]) 3.99 


# print(df_acc.dtypes)
# print(df_acc)

# "asset": "USDT",        //资产
# "walletBalance": "23.72469206",  //余额
# "unrealizedProfit": "0.00000000",  // 未实现盈亏
# "marginBalance": "23.72469206",  // 保证金余额
# "maintMargin": "0.00000000",    // 维持保证金
# "initialMargin": "0.00000000",  // 当前所需起始保证金
# "positionInitialMargin": "0.00000000",  // 持仓所需起始保证金(基于最新标记价格)
# "openOrderInitialMargin": "0.00000000", // 当前挂单所需起始保证金(基于最新标记价格)
# "crossWalletBalance": "23.72469206",  //全仓账户余额
# "crossUnPnl": "0.00000000" // 全仓持仓未实现盈亏
# "availableBalance": "23.72469206",       // 可用余额
# "maxWithdrawAmount": "23.72469206",     // 最大可转出余额
# "marginAvailable": true,   // 是否可用作联合保证金
# "updateTime": 1625474304765  //更新时间

# 账户信息V2 (USER_DATA) GET /fapi/v2/account
# "symbol": "BTCUSDT",  // 交易对
# "initialMargin": "0",   // 当前所需起始保证金(基于最新标记价格)
# "maintMargin": "0", //维持保证金
# "unrealizedProfit": "0.00000000",  // 持仓未实现盈亏
# "positionInitialMargin": "0",  // 持仓所需起始保证金(基于最新标记价格)
# "openOrderInitialMargin": "0",  // 当前挂单所需起始保证金(基于最新标记价格)
# "leverage": "100",  // 杠杆倍率
# "isolated": true,  // 是否是逐仓模式
# "entryPrice": "0.00000",  // 持仓成本价
# "maxNotional": "250000",  // 当前杠杆下用户可用的最大名义价值
# "bidNotional": "0",  // 买单净值，忽略
# "askNotional": "0",  // 卖单净值，忽略
# "positionSide": "BOTH",  // 持仓方向
# "positionAmt": "0",      // 持仓数量
# "updateTime": 0         // 更新时间 


# 用户持仓风险V2 GET /fapi/v2/positionRisk
# "entryPrice": "0.00000", // 开仓均价
# "marginType": "isolated", // 逐仓模式或全仓模式
# "isAutoAddMargin": "false",
# "isolatedMargin": "0.00000000", // 逐仓保证金
# "leverage": "10", // 当前杠杆倍数
# "liquidationPrice": "0", // 参考强平价格
# "markPrice": "6679.50671178",   // 当前标记价格
# "maxNotionalValue": "20000000", // 当前杠杆倍数允许的名义价值上限
# "positionAmt": "0.000", // 头寸数量，符号代表多空方向, 正数为多，负数为空
# "notional": "0",
# "isolatedWallet": "0",
# "symbol": "BTCUSDT", // 交易对
# "unRealizedProfit": "0.00000000", // 持仓未实现盈亏
# "positionSide": "BOTH", // 持仓方向
# "updateTime": 1625474304765   // 更新时间

print( f'{time.strftime("%Y-%m-%d %H:%M:%S")} *****账户 account###可用###分析***** ') 

In [ ]:
###账户 position###持仓###分析###
df_posi = pd.read_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )
df_posi_diff = df_posi[['initialMargin','maintMargin','unrealizedProfit','positionInitialMargin','openOrderInitialMargin','leverage','entryPrice','maxNotional','bidNotional','askNotional','positionAmt','updateTime']].diff(axis=0, periods=1)
df_posi_diff.iloc[0] = 0
df_posi_diff['queryTime'] = df_posi['queryTime'] 
display(df_posi)
print('')
display(df_posi_diff)

df = df_posi.iloc[-1]

print(f'维持保证金 maintMargin = 名义价值 notional * 维持保证金率 maintMarginRatio : {df["maintMargin"]} = { float(df["notional"]) * 0.01 } ')  #ok


# 账户信息V2 (USER_DATA) GET /fapi/v2/account
# "symbol": "BTCUSDT",  // 交易对
# "initialMargin": "0",   // 当前所需起始保证金(基于最新标记价格)
# "maintMargin": "0", //维持保证金
# "unrealizedProfit": "0.00000000",  // 持仓未实现盈亏
# "positionInitialMargin": "0",  // 持仓所需起始保证金(基于最新标记价格)
# "openOrderInitialMargin": "0",  // 当前挂单所需起始保证金(基于最新标记价格)
# "leverage": "100",  // 杠杆倍率
# "isolated": true,  // 是否是逐仓模式
# "entryPrice": "0.00000",  // 持仓成本价
# "maxNotional": "250000",  // 当前杠杆下用户可用的最大名义价值
# "bidNotional": "0",  // 买单净值，忽略
# "askNotional": "0",  // 卖单净值，忽略
# "positionSide": "BOTH",  // 持仓方向
# "positionAmt": "0",      // 持仓数量
# "updateTime": 0         // 更新时间 


# 用户持仓风险V2 GET /fapi/v2/positionRisk
# "entryPrice": "0.00000", // 开仓均价
# "marginType": "isolated", // 逐仓模式或全仓模式
# "isAutoAddMargin": "false",
# "isolatedMargin": "0.00000000", // 逐仓保证金
# "leverage": "10", // 当前杠杆倍数
# "liquidationPrice": "0", // 参考强平价格
# "markPrice": "6679.50671178",   // 当前标记价格
# "maxNotionalValue": "20000000", // 当前杠杆倍数允许的名义价值上限
# "positionAmt": "0.000", // 头寸数量，符号代表多空方向, 正数为多，负数为空
# "notional": "0",
# "isolatedWallet": "0",
# "symbol": "BTCUSDT", // 交易对
# "unRealizedProfit": "0.00000000", // 持仓未实现盈亏
# "positionSide": "BOTH", // 持仓方向
# "updateTime": 1625474304765   // 更新时间

print( f'{time.strftime("%Y-%m-%d %H:%M:%S")} *****账户 position###持仓###分析***** ') 


In [ ]:
###账户 order###订单###分析###
df_ord = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
# df_ord_diff = df_ord[['cumQuote','executedQty','avgPrice','origQty','price','stopPrice','updateTime']].diff(axis=0, periods=1)  #'activatePrice','priceRate',
# df_ord_diff.iloc[0] = 0
# df_ord_diff['time'] = df_ord['time'] 
display(df_ord)
print('')
# display(df_ord_diff)
# print(df_ord.dtypes)

# "clientOrderId": "testOrder", // 用户自定义的订单号
# "cumQty": "0",
# "cumQuote": "0", // 成交金额
# "executedQty": "0", // 成交量
# "orderId": 22542179, // 系统订单号
# "avgPrice": "0.00000",  // 平均成交价
# "origQty": "10", // 原始委托数量
# "price": "0", // 委托价格
# "reduceOnly": false, // 仅减仓
# "side": "SELL", // 买卖方向
# "positionSide": "SHORT", // 持仓方向
# "status": "NEW", // 订单状态
# "stopPrice": "0", // 触发价，对'TRAILING_STOP_MARKET'无效
# "closePosition": false,   // 是否条件全平仓
# "symbol": "BTCUSDT", // 交易对
# "timeInForce": "GTC", // 有效方法
# "type": "TRAILING_STOP_MARKET", // 订单类型
# "origType": "TRAILING_STOP_MARKET",  // 触发前订单类型
# "activatePrice": "9020", // 跟踪止损激活价格, 仅'TRAILING_STOP_MARKET' 订单返回此字段
# "priceRate": "0.3", // 跟踪止损回调比例, 仅'TRAILING_STOP_MARKET' 订单返回此字段
# "updateTime": 1566818724722, // 更新时间
# "workingType": "CONTRACT_PRICE", // 条件价格触发类型
# "priceProtect": false            // 是否开启条件单触发保护

print( f'{time.strftime("%Y-%m-%d %H:%M:%S")} *****账户 order###订单###分析***** ') 


In [ ]:
#账户 持仓风险分析

p1 = {
    "symbol": "BTCUSDT",
    "positionAmt": "0.020",
    "entryPrice": "20198.9",
    "markPrice": "20064.10000000",
    "unRealizedProfit": "-2.69600000",
    "liquidationPrice": "10164.46132879",
    "leverage": "2",
    "maxNotionalValue": "15000000",
    "marginType": "isolated",
    "isolatedMargin": "200.02566569",
    "isAutoAddMargin": "false",
    "positionSide": "BOTH",
    "notional": "401.28200000",
    "isolatedWallet": "202.72166569",
    "updateTime": 1662076800121
}

p2 = {
	"symbol": "BTCUSDT",
	"positionAmt": "0.020",
	"entryPrice": "20198.9",
	"markPrice": "19976.40000000",
	"unRealizedProfit": "-4.45000000",
	"liquidationPrice": "10164.46132879",
	"leverage": "2",
	"maxNotionalValue": "15000000",
	"marginType": "isolated",
	"isolatedMargin": "198.27166569",
	"isAutoAddMargin": "false",
	"positionSide": "BOTH",
	"notional": "399.52800000",
	"isolatedWallet": "202.72166569",
	"updateTime": 1662076800121
}

p3 = {
	"symbol": "BTCUSDT",
	"positionAmt": "0.020",
	"entryPrice": "20198.9",
	"markPrice": "20163.12016894",
	"unRealizedProfit": "-0.71559662",
	"liquidationPrice": "10164.46132879",
	"leverage": "2",
	"maxNotionalValue": "15000000",
	"marginType": "isolated",
	"isolatedMargin": "202.00606907",
	"isAutoAddMargin": "false",
	"positionSide": "BOTH",
	"notional": "403.26240337",
	"isolatedWallet": "202.72166569",
	"updateTime": 1662076800121
}


p4 = {
    "symbol": "BTCUSDT",
    "positionAmt": "0.020",
    "entryPrice": "20198.9",
    "markPrice": "20182.50000000",
    "unRealizedProfit": "-0.32800000",
    "liquidationPrice": "10164.46132879",
    "leverage": "2",
    "maxNotionalValue": "15000000",
    "marginType": "isolated",
    "isolatedMargin": "202.39366569",
    "isAutoAddMargin": "false",
    "positionSide": "BOTH",
    "notional": "403.65000000",
    "isolatedWallet": "202.72166569",
    "updateTime": 1662076800121
}

df1 = p3
df2 = p4

In [ ]:
print(f'标记价格 markPrice 差额:  { float(df2["markPrice"]) - float(df1["markPrice"])  } ')
print(f'逐仓保证金 isolatedMargin 差额:  { float(df2["isolatedMargin"]) - float(df1["isolatedMargin"]) } ')

print(f'逐仓保证金 isolatedMargin 算法差额:  { (float(df2["markPrice"]) - float(df1["markPrice"]) ) * float(df2["positionAmt"]) } ')

print(f'逐仓保证金 isolatedMargin :  { float(df2["isolatedMargin"])  } ')


In [ ]:
print('账户余额 account: ')
df = a1
print(f'钱包余额 walletBalance = marginBalance - unrealizedProfit : {df["walletBalance"]} =  { float(df["marginBalance"]) - float(df["unrealizedProfit"])  } ')  #ok  
print(f'保证金占用 : marginBalance - availableBalance  =  { float(df["marginBalance"]) - float(df["availableBalance"])  } ')    

print('\n持仓 position: ')
df = p5
print(f'当前所需起始保证金(基于最新标记价格) initialMargin :  { float(df["initialMargin"])   } ') 
print(f'标记价格(基于名义价值反推) : markprice = 名义价值 notional / 持仓数量positionAmt  : { float(df["notional"]) / float(df["positionAmt"])  } ')   



In [ ]:
df = p1

print(f'持仓名义价值 notional : {df["notional"]} =  { float(df["positionAmt"]) * float(df["markPrice"])  } ')  #ok  
print(f'保证金余额 Margin Balance : { float(df["positionAmt"]) * float(df["markPrice"]) / float(df["leverage"]) } ')  #ok  

print(f'维持保证金 Maintenance Margin :  { float(df["positionAmt"]) * float(df["markPrice"]) * 0.01 } ')  #ok  杠杆分成 "maintMarginRatio":0.01

print(f'保证金比率 Margin Ratio :  { float(df["positionAmt"]) * float(df["markPrice"]) * 0.01 / (float(df["notional"]) / float(df["leverage"]) ) } ')  #ok  



print(f'isolatedWallet : {df["isolatedWallet"]} =  { float(df["isolatedMargin"]) - float(df["unRealizedProfit"]) } ') #ok

print(f'持仓未实现盈亏 unRealizedProfit : {df["unRealizedProfit"]} =  { (float(df["markPrice"]) - float(df["entryPrice"]) ) * float(df["positionAmt"])  } ')  #ok

print(f'逐仓保证金 isolatedMargin : {df["isolatedMargin"]} =  { float(df["positionAmt"]) * float(df["markPrice"]) / float(df["leverage"]) } , diff : {float(df["isolatedMargin"]) - float(df["notional"]) / float(df["leverage"])  }  ')  #err



